In [1]:
import sys
sys.path.append('./scripts')
sys.path.append('./probvlm')

from tuning_tools import prep_str_args, evaluate_checkpoint, get_evaluation_history, gridsearch, create_base_str_args

[nltk_data] Downloading package punkt to C:\Users\Laptop of
[nltk_data]     Natalie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Laptop of
[nltk_data]     Natalie\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
history_path = './results/eval_ilt.txt'  
model_history = get_evaluation_history(history_path) # Get the history of evaluated models
eval_file = './results/eval_ilt.txt'
split = 'val'

base_str_args = create_base_str_args('baseline', 'ILT', eval_file)
print('base str args:', base_str_args)

# Dictionary of values to gridsearch for hyperparam tuning
gridsearch_dict = {
    '--lr': [5e-4, 5e-5, 5e-6],
    '--epochs' : [30], #list(range(15,36,5)) if 'active-learning' in base_str_args else [35], #[10,15,20,25,30,35],
    '--batch-size' : [16, 32, 64],
    '--label-ratio': [0.1], #[0.05, 0.1, 0.2, 0.4, 0.8],
}

base str args:  
    --train-data ILT
    --val-data ILT
    --save-freq 5
    --eval-file ./results/eval_ilt.txt
    --zeroshot-frequency 5
    --method base
    


In [ ]:
gridsearch(base_str_args, gridsearch_dict, model_history, eval_file, split, change_seed = True)

Number of configs: 9
['--train-data', 'ILT', '--val-data', 'ILT', '--save-freq', '5', '--eval-file', './results/eval_ilt.txt', '--zeroshot-frequency', '5', '--method', 'base', '--lr', '0.0005', '--epochs', '30', '--batch-size', '16', '--label-ratio', '0.1']
Config number 0: 4 repeats
indices d1: [tensor(6), tensor(29), tensor(36), tensor(49), tensor(51)] 5
indices d2: [tensor(0), tensor(1), tensor(2), tensor(3), tensor(4), tensor(5), tensor(7), tensor(8), tensor(9), tensor(10), tensor(11), tensor(12), tensor(13), tensor(14), tensor(15), tensor(16), tensor(17), tensor(18), tensor(19), tensor(20), tensor(21), tensor(22), tensor(23), tensor(24), tensor(25), tensor(26), tensor(27), tensor(28), tensor(30), tensor(31), tensor(32), tensor(33), tensor(34), tensor(35), tensor(37), tensor(38), tensor(39), tensor(40), tensor(41), tensor(42), tensor(43), tensor(44), tensor(45), tensor(46), tensor(47), tensor(48), tensor(50), tensor(52), tensor(53), tensor(54), tensor(55), tensor(56), tensor(57)] 5

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:57<00:00,  1.93s/it]


Evaluation time: 31.042
Evaluation time: 23.124
Evaluation time: 21.67
Evaluation time: 22.129
Evaluation time: 22.028
Evaluation time: 23.404
indices d1: [tensor(1), tensor(24), tensor(48), tensor(55), tensor(57)] 5
indices d2: [tensor(0), tensor(2), tensor(3), tensor(4), tensor(5), tensor(6), tensor(7), tensor(8), tensor(9), tensor(10), tensor(11), tensor(12), tensor(13), tensor(14), tensor(15), tensor(16), tensor(17), tensor(18), tensor(19), tensor(20), tensor(21), tensor(22), tensor(23), tensor(25), tensor(26), tensor(27), tensor(28), tensor(29), tensor(30), tensor(31), tensor(32), tensor(33), tensor(34), tensor(35), tensor(36), tensor(37), tensor(38), tensor(39), tensor(40), tensor(41), tensor(42), tensor(43), tensor(44), tensor(45), tensor(46), tensor(47), tensor(49), tensor(50), tensor(51), tensor(52), tensor(53), tensor(54), tensor(56)] 53


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:51<00:00,  1.72s/it]


Evaluation time: 28.211
